In [10]:
#原本的，不能接受non-singularity matrix的代码
import numpy as np

def algebraic_iterative_proportional_fitting(A, target_row_sums, target_col_sums, epsilon, max_iterations):
    """
    Perform the Algebraic Iterative Proportional Fitting (AIPF) algorithm.

    Parameters:
        A (np.ndarray): Initial matrix of size (n, o).
        target_row_sums (np.ndarray): Target row sums (vector of size n).
        target_col_sums (np.ndarray): Target column sums (vector of size o).
        epsilon (float): Error tolerance for stopping the iteration.
        max_iterations (int): Maximum number of iterations.

    Returns:
        np.ndarray: Adjusted matrix that satisfies the target row and column sums.
    """
    # Initialize diagonal matrices for target row and column sums
    R_diag = np.diag(target_row_sums)
    C_diag = np.diag(target_col_sums)

    # Compute initial row and column sums of A
    row_sums = np.sum(A, axis=1)
    col_sums = np.sum(A, axis=0)

    # Initialize error and iteration counter
    error = np.inf
    iteration = 0

    while error > epsilon and iteration < max_iterations:
        # Scale rows of A to match target row sums
        P_diag = np.diag(row_sums)
        A = np.dot(np.dot(R_diag, np.linalg.inv(P_diag)), A)

        # Scale columns of A to match target column sums
        K_diag = np.diag(col_sums)
        A = np.dot(A, np.dot(np.linalg.inv(K_diag), C_diag))

        # Update row and column sums
        row_sums = np.sum(A, axis=1)
        col_sums = np.sum(A, axis=0)

        # Compute total squared error
        row_error = np.sum((row_sums - target_row_sums) ** 2)
        col_error = np.sum((col_sums - target_col_sums) ** 2)
        error = row_error + col_error

        # Increment iteration counter
        iteration += 1

    return A

# Example usage
if __name__ == "__main__":
    # Initial matrix
    A = np.array([
        [0.5, 1.5, 2.0],
        [2.0, 1.0, 1.0],
        [1.5, 0.5, 1.0]
    ])

    # Target row sums and column sums
    target_row_sums = np.array([4.0, 4.0, 3.0])
    target_col_sums = np.array([3.0, 3.0, 5.0])

    # Parameters
    epsilon = 1e-6
    max_iterations = 100

    # Perform AIPF
    adjusted_matrix = algebraic_iterative_proportional_fitting(A, target_row_sums, target_col_sums, epsilon, max_iterations)

    print("Adjusted Matrix:")
    print(adjusted_matrix)



Adjusted Matrix:
[[0.32478259 1.37662361 2.29905842]
 [1.54353317 1.09040337 1.36578784]
 [1.13168424 0.53297302 1.33515374]]


In [11]:
# 可以接受non-singularity matrix, (3. Element-wise Scaling: Replace the matrix inversion with element-wise operations, which are robust to zeros in the diagonal.)
import numpy as np

def algebraic_iterative_proportional_fitting_singular(A, target_row_sums, target_col_sums, epsilon, max_iterations, delta=1e-9):
    """
    Perform the Algebraic Iterative Proportional Fitting (AIPF) algorithm for singular matrices.

    Parameters:
        A (np.ndarray): Initial matrix of size (n, o).
        target_row_sums (np.ndarray): Target row sums (vector of size n).
        target_col_sums (np.ndarray): Target column sums (vector of size o).
        epsilon (float): Error tolerance for stopping the iteration.
        max_iterations (int): Maximum number of iterations.
        delta (float): Small constant to handle zero or near-zero diagonal entries.

    Returns:
        np.ndarray: Adjusted matrix that satisfies the target row and column sums.
    """
    # Compute initial row and column sums of A
    row_sums = np.sum(A, axis=1)
    col_sums = np.sum(A, axis=0)

    # Initialize error and iteration counter
    error = np.inf
    iteration = 0

    while error > epsilon and iteration < max_iterations:
        # Scale rows of A to match target row sums
        row_scaling_factors = np.where(row_sums > 0, target_row_sums / row_sums, delta)
        A = np.multiply(A.T, row_scaling_factors).T  # Element-wise scaling of rows

        # Scale columns of A to match target column sums
        col_sums = np.sum(A, axis=0)
        col_scaling_factors = np.where(col_sums > 0, target_col_sums / col_sums, delta)
        A = np.multiply(A, col_scaling_factors)  # Element-wise scaling of columns

        # Update row and column sums
        row_sums = np.sum(A, axis=1)
        col_sums = np.sum(A, axis=0)

        # Compute total squared error
        row_error = np.sum((row_sums - target_row_sums) ** 2)
        col_error = np.sum((col_sums - target_col_sums) ** 2)
        error = row_error + col_error

        # Increment iteration counter
        iteration += 1

    return A

# Example usage
if __name__ == "__main__":
    # Initial matrix (singular or non-singular)
    A = np.array([
        [0.5, 1.5, 0.0],
        [2.0, 0.0, 0.0],
        [1.5, 0.5, 1.0]
    ])

    # Target row sums and column sums
    target_row_sums = np.array([4.0, 4.0, 3.0])
    target_col_sums = np.array([3.0, 3.0, 5.0])

    # Parameters
    epsilon = 1e-6
    max_iterations = 100

    # Perform AIPF for singular matrix
    adjusted_matrix = algebraic_iterative_proportional_fitting_singular(A, target_row_sums, target_col_sums, epsilon, max_iterations)

    print("Adjusted Matrix:")
    print(adjusted_matrix)


Adjusted Matrix:
[[1.47399340e-02 3.00000000e+00 0.00000000e+00]
 [2.98526007e+00 0.00000000e+00 0.00000000e+00]
 [3.57370629e-36 8.08168768e-35 5.00000000e+00]]


In [12]:
# 可以接受non-singularity matrix, (2. regularization. e.g.: Add a small constant to the diagonal elements of the row and column scaling matrices to make them invertible.)

import numpy as np

def algebraic_iterative_proportional_fitting(A, target_row_sums, target_col_sums, epsilon, max_iterations, regularization=1e-8):
    """
    Perform the Algebraic Iterative Proportional Fitting (AIPF) algorithm with regularization for singular matrices.

    Parameters:
        A (np.ndarray): Initial matrix of size (n, o).
        target_row_sums (np.ndarray): Target row sums (vector of size n).
        target_col_sums (np.ndarray): Target column sums (vector of size o).
        epsilon (float): Error tolerance for stopping the iteration.
        max_iterations (int): Maximum number of iterations.
        regularization (float): Small constant added to diagonal elements for regularization.

    Returns:
        np.ndarray: Adjusted matrix that satisfies the target row and column sums.
    """
    # Initialize diagonal matrices for target row and column sums
    R_diag = np.diag(target_row_sums)
    C_diag = np.diag(target_col_sums)

    # Compute initial row and column sums of A
    row_sums = np.sum(A, axis=1)
    col_sums = np.sum(A, axis=0)

    # Initialize error and iteration counter
    error = np.inf
    iteration = 0

    while error > epsilon and iteration < max_iterations:
        # Scale rows of A to match target row sums, with regularization
        P_diag = np.diag(row_sums + regularization)
        A = np.dot(np.dot(R_diag, np.linalg.inv(P_diag)), A)

        # Scale columns of A to match target column sums, with regularization
        K_diag = np.diag(col_sums + regularization)
        A = np.dot(A, np.dot(np.linalg.inv(K_diag), C_diag))

        # Update row and column sums
        row_sums = np.sum(A, axis=1)
        col_sums = np.sum(A, axis=0)

        # Compute total squared error
        row_error = np.sum((row_sums - target_row_sums) ** 2)
        col_error = np.sum((col_sums - target_col_sums) ** 2)
        error = row_error + col_error

        # Increment iteration counter
        iteration += 1

    return A

# Example usage
if __name__ == "__main__":
    # Initial matrix
    A = np.array([
        [0.5, 1.5, 2.0],
        [2.0, 1.0, 1.0],
        [1.5, 0.5, 1.0]
    ])

    # Target row sums and column sums
    target_row_sums = np.array([4.0, 4.0, 3.0])
    target_col_sums = np.array([3.0, 3.0, 5.0])

    # Parameters
    epsilon = 1e-6
    max_iterations = 100
    regularization = 1e-8

    # Perform AIPF
    adjusted_matrix = algebraic_iterative_proportional_fitting(A, target_row_sums, target_col_sums, epsilon, max_iterations, regularization)

    print("Adjusted Matrix:")
    print(adjusted_matrix)


Adjusted Matrix:
[[0.32478259 1.3766236  2.29905841]
 [1.54353316 1.09040336 1.36578783]
 [1.13168423 0.53297302 1.33515373]]


In [13]:
# 可以接受non-singularity matrix, (1. Handle Zero Diagonal Entries: Instead of directly inverting 𝑃 or 𝐾, check for zero diagonal entries and replace them with a small positive value (a regularization term) to avoid singularity.)

import numpy as np

def algebraic_iterative_proportional_fitting(A, target_row_sums, target_col_sums, epsilon, max_iterations, regularization=1e-10):
    """
    Perform the Algebraic Iterative Proportional Fitting (AIPF) algorithm.

    Parameters:
        A (np.ndarray): Initial matrix of size (n, o).
        target_row_sums (np.ndarray): Target row sums (vector of size n).
        target_col_sums (np.ndarray): Target column sums (vector of size o).
        epsilon (float): Error tolerance for stopping the iteration.
        max_iterations (int): Maximum number of iterations.
        regularization (float): Small positive value to handle zero diagonal entries.

    Returns:
        np.ndarray: Adjusted matrix that satisfies the target row and column sums.
    """
    # Initialize diagonal matrices for target row and column sums
    R_diag = np.diag(target_row_sums)
    C_diag = np.diag(target_col_sums)

    # Compute initial row and column sums of A
    row_sums = np.sum(A, axis=1)
    col_sums = np.sum(A, axis=0)

    # Initialize error and iteration counter
    error = np.inf
    iteration = 0

    while error > epsilon and iteration < max_iterations:
        # Handle zero diagonal entries in row sums
        row_sums = np.where(row_sums == 0, regularization, row_sums)
        P_diag = np.diag(row_sums)
        A = np.dot(np.dot(R_diag, np.linalg.inv(P_diag)), A)

        # Update column sums and handle zero diagonal entries
        col_sums = np.sum(A, axis=0)
        col_sums = np.where(col_sums == 0, regularization, col_sums)
        K_diag = np.diag(col_sums)
        A = np.dot(A, np.dot(np.linalg.inv(K_diag), C_diag))

        # Update row sums
        row_sums = np.sum(A, axis=1)

        # Compute total squared error
        row_error = np.sum((row_sums - target_row_sums) ** 2)
        col_error = np.sum((col_sums - target_col_sums) ** 2)
        error = row_error + col_error

        # Increment iteration counter
        iteration += 1

    return A

# Example usage
if __name__ == "__main__":
    # Initial matrix
    A = np.array([
        [0.5, 1.5, 2.0],
        [2.0, 1.0, 1.0],
        [1.5, 0.5, 1.0]
    ])

    # Target row sums and column sums
    target_row_sums = np.array([4.0, 4.0, 3.0])
    target_col_sums = np.array([3.0, 3.0, 5.0])

    # Parameters
    epsilon = 1e-6
    max_iterations = 100

    # Perform AIPF
    adjusted_matrix = algebraic_iterative_proportional_fitting(A, target_row_sums, target_col_sums, epsilon, max_iterations)

    print("Adjusted Matrix:")
    print(adjusted_matrix)


Adjusted Matrix:
[[0.32472972 1.3764872  2.29883222]
 [1.54356753 1.09049711 1.36590621]
 [1.13170274 0.53301569 1.33526157]]
